# SOFTMAX REGRESSION FROM SCRATCH

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import sklearn
from sklearn.datasets import fetch_openml

# IMPORT MNIST DATA
mnist = fetch_openml("mnist_784", version=1)
X, y = np.array(mnist["data"]), np.array(mnist["target"])


In [ ]:
# SPLIT DATA INTO TRAINING AND TESTING SETS
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]



In [ ]:
# NORMALIZE DATA
X_train = X_train / 255
X_test = X_test / 255



In [ ]:
# todo: Complete this cell and continue in the same format below
# SOFTMAX FUNCTION
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exp_sums

# CROSS-ENTROPY LOSS FUNCTION
def cross_entropy_loss(y, y_proba):
    m = len(y)
    loss = -np.mean(np.sum(y * np.log(y_proba + 1e-7), axis=1))
    return loss

# CREATE SOFTMAX REGRESSION MODEL FROM SCRATCH
class MySoftmaxRegression:
    def __init__(self, n_inputs, n_outputs):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.theta = np.random.randn(n_inputs, n_outputs)
    
    def predict(self, X):
        logits = X.dot(self.theta)
        return softmax(logits)
    
    def fit(self, X, y, learning_rate=0.1, n_iterations=1000):
        m = len(X)
        Y = np.eye(self.n_outputs)[y.astype(int)]
        
        for iteration in range(n_iterations):
            y_proba = self.predict(X)
            loss = cross_entropy_loss(Y, y_proba)
            error = y_proba - Y
            gradients = 1/m * X.T.dot(error)
            self.theta = self.theta - learning_rate * gradients
            if iteration % 100 == 0:
                print(iteration, loss)
